In [40]:
import psycopg2
import json
import geojson
import tempfile
from django.db import transaction
import pandas as pd

In [207]:
conn.rollback()
cur.execute('DROP TABLE "tweets";')
conn.commit()

In [206]:
tweets = pd.read_json('../../data/tweets_1M.json','r')
tweets = tweets[tweets['lat']>30]

In [161]:
# Duplicate SQL connection using sqlalchemy to take advantage of 
# pandas built in "to_sql" method

from sqlalchemy import create_engine
enginestring = 'postgresql://{}:{}@localhost:5432/template_postgis'.format('postgres', 'teresa')
engine = create_engine(enginestring)

In [185]:
connection = engine.connect()
result = connection.execute("SELECT srtext FROM spatial_ref_sys WHERE srid = 32610;")
for row in result:
    print (row)

('PROJCS["WGS 84 / UTM zone 10N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-123],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32610"]]',)


In [208]:
tweets.to_sql('tweets', engine)

In [188]:
conn = psycopg2.connect("dbname=template_postgis user=postgres host=localhost password=teresa")
cur = conn.cursor()
#cur.execute("""SELECT srtext FROM spatial_ref_sys WHERE srid = 32610;""")
#rows = cur.fetchall()

In [189]:
cur.execute("SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'tweets';")
cur.fetchall()

[('index', 'bigint'),
 ('id', 'bigint'),
 ('lat', 'double precision'),
 ('lng', 'double precision'),
 ('text', 'text'),
 ('timeStamp', 'timestamp without time zone'),
 ('user_id', 'bigint')]

In [209]:
# New column for geometry
cur.execute("SELECT AddGeometryColumn ('tweets','location',4326,'POINT',2);")

In [210]:
cur.execute("UPDATE tweets SET location = ST_SetSRID(ST_Point(lng, lat), 4326);")
conn.commit()

In [73]:
# bash command to insert shapefile into postgis database
# shp2pgsql -I -W "latin1" -s 4326  /Users/yiyange/Desktop/CE\ 263N/data/tl_2010_06_county10/tl_2010_06_county10.shp public.ca_census_tract | psql -U postgres -d template_postgis

# Check data types in the counties 
# cur.execute("SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'public.ca_census_tract';")
# cur.fetchall()
#???

[]

In [ ]:
cur.execute("SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'counties';")
cur.fetchall()

In [192]:
cur.execute("SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'tweets';")
cur.fetchall()

[('index', 'bigint'),
 ('id', 'bigint'),
 ('lat', 'double precision'),
 ('lng', 'double precision'),
 ('text', 'text'),
 ('timeStamp', 'timestamp without time zone'),
 ('user_id', 'bigint'),
 ('location', 'USER-DEFINED')]

In [211]:
# Calculate the number of tweets inside of Contra Costa County = 06013.

conn.rollback()
query = "SELECT count(*) FROM counties, tweets \
         WHERE counties.geoid10='06013' \
         AND ST_Intersects(counties.geom, tweets.location);"

cur.execute(query)
cur.fetchall()

[(85141,)]

In [214]:
# How many Tweets fall 100 miles outside of Alameda County
conn.rollback()
query = "SELECT count(*) FROM counties, tweets \
        WHERE counties.geoid10='06001' \
        AND NOT ST_Dwithin(ST_Transform(counties.geom, 26910), ST_Transform(tweets.location, 26910), 160934);"
cur.execute(query)
cur.fetchall()

InternalError: transform: couldn't project point (-6.85488 33.9534 0): latitude or longitude exceeded limits (-14)


In [135]:
conn.rollback()

In [139]:
#query = 'ALTER TABLE "ca_census_data" RENAME COLUMN "pop100200" TO "pop1002000"'
#cur.execute(query)
cur.execute("SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'ca_census_data';")
cur.fetchall()

[('geoid', 'character varying'),
 ('sumlev', 'character varying'),
 ('state', 'character varying'),
 ('county', 'character varying'),
 ('cbsa', 'character varying'),
 ('csa', 'character varying'),
 ('necta', 'integer'),
 ('cnecta', 'integer'),
 ('name', 'character varying'),
 ('pop100', 'integer'),
 ('hu100', 'integer'),
 ('pop1002000', 'integer'),
 ('hu1002000', 'integer'),
 ('p001001', 'integer'),
 ('p0010012000', 'integer')]

In [138]:
conn.commit()

In [198]:
conn.rollback()
query = """
SELECT 

county_pop.name, 
county_tweets.count_tweets::float/county_pop.pop::float

FROM

    (SELECT counties.name10 as name, ca_census_data.pop1002000 as pop, counties.geom as geom 
    FROM ca_census_data 
    INNER JOIN counties 
    ON counties.geoid10 = ca_census_data.geoid) county_pop
    
    INNER JOIN

    (SELECT counties.name10 as name, count(*) as count_tweets 
    FROM tweets, counties
    WHERE ST_Intersects(tweets.location, counties.geom)
    GROUP BY counties.name10) county_tweets

ON county_pop.name = county_tweets.name;

"""

cur.execute(query)
result = cur.fetchall()

In [199]:
df = pd.DataFrame(result, columns=['NAME10', 'tweets_per_capita'])
df.head()

,NAME10,tweets_per_capita
0,Orange,0.002740
1,Tehama,0.045914
2,Santa Barbara,0.001530
3,Monterey,0.016467
4,Placer,0.002685
